In [4]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [6]:
ds = datasets.FashionMNIST(
    root='data',
    train= True,
    download= True,
    transform= ToTensor(),
    target_transform= Lambda(lambda y: torch.zeros(10, dtype= torch.float).scatter_(0, torch.tensor(y), value=1))
)

100.0%
100.0%
100.0%
100.0%


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


In [8]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
X = torch.rand(1, 28, 28)
model = NeuralNetwork()
pred_prob = model(X)
y_pred = torch.argmax(pred_prob, dim=1)
print(f'Predicted class: {y_pred.item()}')

Predicted class: 4


In [18]:
def training_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y
        pred = model(X)
        loss = loss_fn(pred, y)
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()
train_dataloader = torch.utils.data.DataLoader(ds, batch_size= 64, shuffle= True)
test_dataloader = torch.utils.data.DataLoader(ds, batch_size= 64, shuffle= False)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    training_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)  

Epoch 1
-------------------------------
loss: 1.464138  [    0/60000]
loss: 1.471298  [ 6400/60000]
loss: 1.471298  [ 6400/60000]
loss: 1.339304  [12800/60000]
loss: 1.339304  [12800/60000]
loss: 1.419005  [19200/60000]
loss: 1.419005  [19200/60000]
loss: 1.425878  [25600/60000]
loss: 1.425878  [25600/60000]
loss: 1.286510  [32000/60000]
loss: 1.286510  [32000/60000]
loss: 1.306510  [38400/60000]
loss: 1.306510  [38400/60000]
loss: 1.202268  [44800/60000]
loss: 1.202268  [44800/60000]
loss: 1.309562  [51200/60000]
loss: 1.309562  [51200/60000]
loss: 1.264597  [57600/60000]
loss: 1.264597  [57600/60000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.234596 

Epoch 2
-------------------------------
loss: 1.398934  [    0/60000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.234596 

Epoch 2
-------------------------------
loss: 1.398934  [    0/60000]
loss: 1.215236  [ 6400/60000]
loss: 1.215236  [ 6400/60000]
loss: 1.176357  [12800/60000]
loss: 1.176357  [12800/60000]
loss: 1.082236  [19200/60000

In [20]:
torch.save(model.state_dict(), 'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
